In [1]:
from SourceFiles.teproteus import TEProteusAdmin as TepAdmin
import matplotlib.pyplot as plt
import numpy as np
import time

# connect and initialize
admin = TepAdmin() #required to control PXI module
sid = 8 #PXI slot WDS found
inst = admin.open_instrument(slot_id=sid)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp) # Print *IDN
inst.send_scpi_cmd('*CLS; *RST')

Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
connected to: Tabor Electronics,P9484M,000002243278,1.237.0 --slot#: 8,


0

In [2]:
#Set rates for DAC
sampleRateDAC = 1.125E9
#Set CF
ncoFreq= 50E6

#wavefore parameters
max_dac=65535 # Max Dac
half_dac=max_dac/2 # DC Level
data_type = np.uint16 # DAC data type
onLen = 32 * 20 # Pulse on time multiple of 32 or 28ns

segTime = onLen / (sampleRateDAC)
print('On time')
print(segTime)
dacWaveOn_I = np.ones(onLen)
dacWaveOn_I = dacWaveOn_I * max_dac
dacWaveOn_I = dacWaveOn_I.astype(data_type)
dacWaveOn_Q = dacWaveOn_I
# reshapes for IQIQIQIQIQ....
arr_tuple = (dacWaveOn_I, dacWaveOn_Q)
dacWaveOn_IQ = np.vstack(arr_tuple).reshape((-1,), order='F')
offLen = 32
dacWaveOff_I = np.ones(offLen)
dacWaveOff_I = dacWaveOff_I * half_dac
dacWaveOff_I = dacWaveOff_I.astype(data_type)
dacWaveOff_Q = dacWaveOff_I
# reshapes for IQIQIQIQIQ....
arr_tuple = (dacWaveOff_I, dacWaveOff_Q)
dacWaveOff_IQ = np.vstack(arr_tuple).reshape((-1,), order='F')

On time
5.688888888888889e-07


In [3]:
ch = 1 # everythinf after relates to CH 1
cmd = ':INST:CHAN {0}'.format(ch)
inst.send_scpi_cmd(cmd)
cmd = ':FREQ:RAST {0}'.format(2.5E9) # force to max 16 bit DAC
inst.send_scpi_cmd(cmd)
inst.send_scpi_cmd(':INIT:CONT OFF')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TRIG:SOURce:ENAB {}'.format('TRG1'))
inst.send_scpi_cmd(':TRIG:SELECT {}'.format('TRG1'))
inst.send_scpi_cmd(':TRIG:LEV 0.5')
inst.send_scpi_cmd(':TRIG:DEL 0E-9')
inst.send_scpi_cmd(':TRIG:SLOP POS')
inst.send_scpi_cmd(':TRIG:COUP ON')
inst.send_scpi_cmd(':TRIGger:STAT ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print("End of trig Error = ")
print(resp)

End of trig Error = 
0, no error


In [4]:
segnum = 1
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, len(dacWaveOn_IQ))
inst.send_scpi_cmd(cmd)
# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)
# Increase the timeout before writing binary-data:
inst.timeout = 30000
# Send the binary-data with *OPC? added to the beginning of its prefix.
inst.write_binary_data('*OPC?; :TRAC:DATA', dacWaveOn_IQ)
# Set normal timeout
inst.timeout = 10000
resp = inst.send_scpi_query(':SYST:ERR?')
print("Trace Download Error = ")
print(resp)

Trace Download Error = 
0, no error


In [5]:
segnum = 2
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, len(dacWaveOff_IQ))
inst.send_scpi_cmd(cmd)
# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)
# Increase the timeout before writing binary-data:
inst.timeout = 30000
# Send the binary-data with *OPC? added to the beginning of its prefix.
inst.write_binary_data('*OPC?; :TRAC:DATA', dacWaveOff_IQ)
# Set normal timeout
inst.timeout = 10000
resp = inst.send_scpi_query(':SYST:ERR?')
print("Trace Download Error = ")
print(resp)

Trace Download Error = 
0, no error


In [8]:
cmd = ':FREQ:RAST {0}'.format(2.5E9)  
inst.send_scpi_cmd(cmd)
cmd = ':SOUR:MODE DUC'
inst.send_scpi_cmd(cmd)
cmd = ':SOUR:INT X8'
resp = inst.send_scpi_cmd(cmd)
cmd = ':SOUR:IQM ONE'
resp = inst.send_scpi_cmd(cmd)
sampleRateDACInt = sampleRateDAC * 8
print('Interpolated Sample Clk Freq {0}'.format(sampleRateDACInt))
cmd = ':FREQ:RAST {0}'.format(sampleRateDACInt)
resp = inst.send_scpi_cmd(cmd)
resp = inst.send_scpi_query(':SYST:ERR?')
print("IQ Set Error = ")
print(resp)
cmd = ':SOUR:NCO:CFR1 {0}'.format(ncoFreq)
resp = inst.send_scpi_cmd(cmd)

Interpolated Sample Clk Freq 9000000000.0
IQ Set Error = 
0, no error
